<a href="https://colab.research.google.com/github/boodie04/Supervised_ML/blob/main/Brain_Tumor_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score
from PIL import Image
from imblearn.over_sampling import SMOTE


In [17]:
from google.colab import files
uploaded = files.upload()  # upload Dataset.rar

!apt-get install -y unrar
!unrar x Dataset.rar /content/Dataset/


Saving Dataset.rar to Dataset (1).rar
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from Dataset.rar

Creating    /content/Dataset                                          OK
Creating    /content/Dataset/Dataset                                  OK
Creating    /content/Dataset/Dataset/Brain Tumor Data Set             OK
Creating    /content/Dataset/Dataset/Brain Tumor Data Set/Grayscale_Brain_Healthy  OK
Extracting  /content/Dataset/Dataset/Brain Tumor Data Set/Grayscale_Brain_Healthy/Not Cancer  (1).jpeg       0%  OK 
Extracting  /content/Dataset/Dataset/Brain Tumor Data Set/Grayscale_Brain_Healthy/Not Cancer  (1).jpg       0%  OK 
Extracting  /content/Dataset/Dataset/Brain Tumor Data Set/Grayscale_Brain_Healthy/

In [28]:
import os

base_path = '/content/Dataset'
print("Exists?", os.path.exists(base_path))
print("Contents of base folder:", os.listdir(base_path))
print(os.listdir('/content/Dataset'))



Exists? True
Contents of base folder: ['Dataset']
['Dataset']


In [29]:
def load_images(folder_path, image_size=(100, 100)):
    X = []
    y = []

    for label, category in enumerate(['notumor', 'tumor']):
        category_path = os.path.join(folder_path, category)
        for filename in os.listdir(category_path):
            img_path = os.path.join(category_path, filename)
            try:
                img = Image.open(img_path).convert('L')  # Grayscale
                img = img.resize(image_size)
                X.append(np.array(img).flatten())
                y.append(label)
            except:
                continue
    return np.array(X), np.array(y)
import os

X, y = load_images('/content/Dataset/Dataset/Dataset')  # Change path as needed
print(f"Loaded {len(X)} images.")


Loaded 4600 images.


Split DataSet

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


Apply PCA 100 components

In [31]:
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


Train

In [32]:
def evaluate_models(X_train, X_test, y_train, y_test):
    results = []
    ks = [1, 3, 5, 7]

    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k)
        start = time.time()
        knn.fit(X_train, y_train)
        end = time.time()
        acc = knn.score(X_test, y_test)
        results.append(("KNN", k, acc, end - start, "-"))

    dt = DecisionTreeClassifier()
    start = time.time()
    dt.fit(X_train, y_train)
    end = time.time()
    acc = dt.score(X_test, y_test)
    results.append(("DT", "-", acc, end - start, "-"))

    svm = SVC()
    start = time.time()
    svm.fit(X_train, y_train)
    end = time.time()
    acc = svm.score(X_test, y_test)
    sv = np.sum(svm.n_support_)
    results.append(("SVM", "-", acc, end - start, sv))

    print("\nClassifier Results (Original Set):")
    print("Model\tK\tAccuracy\tTime\tSupport Vectors")
    for model in results:
        print(f"{model[0]}\t{model[1]}\t{model[2]:.4f}\t{model[3]:.2f}s\t{model[4]}")

    return results

original_results = evaluate_models(X_train_pca, X_test_pca, y_train, y_test)



Classifier Results (Original Set):
Model	K	Accuracy	Time	Support Vectors
KNN	1	0.9674	0.00s	-
KNN	3	0.9232	0.00s	-
KNN	5	0.8964	0.00s	-
KNN	7	0.8768	0.00s	-
DT	-	0.8928	0.81s	-
SVM	-	0.9326	0.77s	1648


Apply DSCAN

def apply_dbscan_classwise(X, y, eps=5, min_samples=3):
    reduced_X = []
    reduced_y = []

    for label in np.unique(y):
        class_X = X[y == label]
        clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(class_X)
        border_idx = clustering.labels_ == 0  # label 0 often represents border
        reduced_X.extend(class_X[border_idx])
        reduced_y.extend([label] * np.sum(border_idx))
    
    return np.array(reduced_X), np.array(reduced_y)

X_train_reduced, y_train_reduced = apply_dbscan_classwise(X_train_pca, y_train)
print(f"Reduced training samples: {len(X_train_reduced)}")


In [93]:
import numpy as np
from sklearn.cluster import DBSCAN

def apply_dbscan_classwise(X, y, eps=30, min_samples=3):
    """
    Applies DBSCAN clustering separately on each class in the dataset,
    and extracts the border points (non-core but clustered points) from each class.
    This is for numerosity reduction while preserving class structure.

    Parameters:
    -----------
    X : np.ndarray
        Feature matrix (e.g., PCA-transformed features)
    y : np.ndarray
        Corresponding class labels for each sample in X
    eps : float
        The maximum distance between two samples for one to be considered
        as in the neighborhood of the other (DBSCAN parameter)
    min_samples : int
        The number of samples in a neighborhood for a point to be considered
        as a core point (DBSCAN parameter)

    Returns:
    --------
    reduced_X : np.ndarray
        The reduced feature matrix containing border points only from each class
    reduced_y : np.ndarray
        The corresponding labels for the reduced feature matrix
    """

    reduced_X = []  # List to store border points across classes
    reduced_y = []  # Corresponding labels for reduced_X
    print(np.unique(y),"np.unique(y)")

    # Loop over each unique class label separately
    for label in np.unique(y):
        print(label,"<-- ite num")

        # Select samples belonging to the current class
        class_X = X[y == label]

        # Apply DBSCAN clustering to the samples of this class
        clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(class_X)

        # Create a boolean mask for core points (samples that form cluster centers)
        core_samples_mask = np.zeros_like(clustering.labels_, dtype=bool)
        #print(core_samples_mask)
        core_samples_mask[clustering.core_sample_indices_] = True
        #print(core_samples_mask)
        # Border points are those that are:
        # 1) Assigned to a cluster (label != -1, so not noise)what
        # 2) Not core points (border samples lie on the edge of clusters)

       #--> border_mask = (clustering.labels_ != -1) & (~core_samples_mask)

        reduced_data_mask_no_noise_only= (clustering.labels_ != -1)
        #print(border_mask,"boreder mask test")
        # Extract the actual border points using the mask

        #--> border_points = class_X[border_mask]

        border_points_and_core = class_X[reduced_data_mask_no_noise_only]


        # Print statistics for debugging and monitoring
        print(f"Class {label}: Total samples={len(class_X)}, Core points={core_samples_mask.sum()}, Border points + core={border_points_and_core.shape[0]}")

        # Append border points and their class labels to the output lists
        reduced_X.extend(border_points_and_core)
        reduced_y.extend([label] * len(border_points_and_core))
        '''
        reduced_X.extend(border_points)
        Adds all border_points (a list or array of points) to the end of the list reduced_X.

        reduced_y.extend([label] * len(border_points))
        Creates a list with the current label repeated for each border point (len(border_points) times).
        Adds these labels to the end of reduced_y.

        '''

    # Convert lists to numpy arrays before returning
    return np.array(reduced_X), np.array(reduced_y)

X_train_reduced, y_train_reduced = apply_dbscan_classwise(X_train_pca, y_train)
print(f"Reduced training samples: {len(X_train_reduced)}")
for label in np.unique(y):
      print(label)
      print(X_train_reduced[y_train_reduced == label])


[0 1] np.unique(y)
0 <-- ite num
Class 0: Total samples=1461, Core points=395, Border points + core=419
1 <-- ite num
Class 1: Total samples=1759, Core points=42, Border points + core=42
Reduced training samples: 461
0
[[-1549.47636396  -978.46280929  -243.15503643 ...  -689.18964343
    747.88964826  -173.77508003]
 [-1444.78530174 -1944.38479311   949.46710267 ...   292.27558138
    -51.2504446   1112.39818822]
 [ 2416.93301679 -1527.51847228  -242.81153236 ...  -177.37760242
   -344.92838673  -835.0479041 ]
 ...
 [  663.61567719 -2162.1282844  -1170.09090628 ...    92.47637686
   1052.5138443    693.27416805]
 [  439.40002484 -2485.81435722   249.4987642  ...  -490.50100672
   -473.04302783  -488.97212822]
 [  631.18006269   560.63477864    18.91473595 ...  -241.999465
   -589.78234192  -267.37447333]]
1
[[ 2589.87215968  2249.61055483  1532.66613493   821.98471485
    570.75489464  -793.75615313  1791.67179363   590.80492877
    226.06470343    73.43238145]
 [  609.82132142  2613.2

Test The DBSCAN

NameError: name 'border_mask' is not defined

Retrain on Reduced

In [100]:
def evaluate_and_speedup(X_full, y_full, X_reduced, y_reduced, X_test, y_test):
    print("Reduced training labels distribution:", np.bincount(y_reduced))

    classifiers = {
        "KNN-1": KNeighborsClassifier(n_neighbors=1),
        "KNN-3": KNeighborsClassifier(n_neighbors=3),
        "KNN-5": KNeighborsClassifier(n_neighbors=5),
        "KNN-5": KNeighborsClassifier(n_neighbors=7),
        "DT": DecisionTreeClassifier(),
        "SVM": SVC()
    }

    results = []

    print("\n--- Retraining on Full vs Reduced Sets ---")
    for name, clf in classifiers.items():
        # Train on full data
        start_full = time.time()
        clf.fit(X_full, y_full)
        time_full = time.time() - start_full

        # Train on reduced data
        start_reduced = time.time()
        clf.fit(X_reduced, y_reduced)
        time_reduced = time.time() - start_reduced

        # Speedup calculation
        speedup = time_full / time_reduced if time_reduced > 0 else float('inf')

        # Test accuracy on test set (always same test set)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        # Support vectors count for SVM
        sv_count = clf.n_support_.sum() if name == "SVM" else "-"

        print(f"{name}: Full Train Time={time_full:.2f}s, Reduced Train Time={time_reduced:.2f}s, Speedup={speedup:.2f}x, Accuracy={acc:.4f}, Support Vectors={sv_count}")

        results.append({
            "Model": name,
            "TrainTimeFull": time_full,
            "TrainTimeReduced": time_reduced,
            "Speedup": speedup,
            "Accuracy": acc,
            "SupportVectors": sv_count
        })

    return results

# Usage
reduced_results = evaluate_and_speedup(X_train_pca, y_train, X_train_reduced, y_train_reduced, X_test_pca, y_test)


Reduced training labels distribution: [336  32]

--- Retraining on Full vs Reduced Sets ---
KNN-1: Full Train Time=0.00s, Reduced Train Time=0.00s, Speedup=2.13x, Accuracy=0.4971, Support Vectors=-
KNN-3: Full Train Time=0.00s, Reduced Train Time=0.00s, Speedup=2.12x, Accuracy=0.4971, Support Vectors=-
KNN-5: Full Train Time=0.00s, Reduced Train Time=0.00s, Speedup=2.18x, Accuracy=0.4906, Support Vectors=-
DT: Full Train Time=0.33s, Reduced Train Time=0.01s, Speedup=33.92x, Accuracy=0.4775, Support Vectors=-
SVM: Full Train Time=0.45s, Reduced Train Time=0.00s, Speedup=130.74x, Accuracy=0.4652, Support Vectors=74


In [101]:
all_results = []
for dim in range(10, 101, 10):
    pca = PCA(n_components=dim)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    X_train_reduced, y_train_reduced = apply_dbscan_classwise(X_train_pca, y_train)

    smote = SMOTE()
    X_balanced, y_balanced = smote.fit_resample(X_train_reduced, y_train_reduced)

    results = evaluate_models(X_balanced, X_test_pca, y_balanced, y_test)

    for model in results:
        all_results.append((dim, *model))

# Optional: Save to CSV or plot
import pandas as pd
df = pd.DataFrame(all_results, columns=["PCA_Dim", "Model", "K", "Accuracy", "Time", "Support_Vectors"])
df.to_csv("summary_results.csv", index=False)


[0 1] np.unique(y)
0 <-- ite num
Class 0: Total samples=1461, Core points=395, Border points + core=419
1 <-- ite num
Class 1: Total samples=1759, Core points=42, Border points + core=42

Classifier Results (Original Set):
Model	K	Accuracy	Time	Support Vectors
KNN	1	0.5362	0.00s	-
KNN	3	0.5304	0.00s	-
KNN	5	0.5283	0.00s	-
KNN	7	0.5572	0.00s	-
DT	-	0.5203	0.01s	-
SVM	-	0.5130	0.02s	187
[0 1] np.unique(y)
0 <-- ite num
Class 0: Total samples=1461, Core points=361, Border points + core=380
1 <-- ite num
Class 1: Total samples=1759, Core points=38, Border points + core=38

Classifier Results (Original Set):
Model	K	Accuracy	Time	Support Vectors
KNN	1	0.5667	0.00s	-
KNN	3	0.5630	0.00s	-
KNN	5	0.5645	0.00s	-
KNN	7	0.5790	0.00s	-
DT	-	0.5341	0.01s	-
SVM	-	0.4942	0.03s	168
[0 1] np.unique(y)
0 <-- ite num
Class 0: Total samples=1461, Core points=352, Border points + core=362
1 <-- ite num
Class 1: Total samples=1759, Core points=35, Border points + core=35

Classifier Results (Original Set):
M